In [95]:
import numpy as np
import librosa, os, keras, time
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pydub.utils import make_chunks
from collections import deque

In [96]:
dir1 = './Instrument-Classification/IRMAS-TestingData-Part1/Part1/'
dir2 = './Instrument-Classification/IRMAS-TestingData-Part2/Part2/'
dir3 = './Instrument-Classification/IRMAS-TestingData-Part3/Part3/'

#model = keras.models.load_model('ICmodel_pggbvv.h5')
model = keras.models.load_model('ICmodel_cross.h5')

dir_array = [dir1,dir2,dir3]
#instr = ['pia','gac','gel','buz','vio','voi']
instr = ['pia','gac']

In [97]:
def to_categorical(list_final):
    temp_list = [[0]*len(instr)]*(len(list_final))
    for i in range(len(list_final)):
        x = [0]*len(instr)
        x.insert(0,1)
        x.pop()
        for ins in instr:
            if (ins in list_final[i]): 
                temp_list[i] = [sum(x) for x in zip(temp_list[i],x)]
            x.insert(0,x.pop())
    return temp_list[0]

def prediction(predict):
    prediction = model.predict(predict)
    sum = np.zeros(len(prediction[0]))
    for pred in prediction:
        for i in range(len(pred)):
            sum[i] = sum[i] + pred[i]
    sum[:] = [round(x/len(prediction),2) for x in sum]
    return sum

def average_accuracy(y_pred,y_label):
    acc = 0
    for i in range(len(y_pred)):
        acc = acc + abs(y_pred[i]-y_label[i])
    acc = acc / len(y_pred)
    return round(100*(1-acc),2)

def existance_accuracy(y_pred,y_label):
    acc = 0
    for i in range(len(y_pred)):
        if (y_pred[i]>=0.5 and y_label[i]==1): acc+=1
        #elif (y_pred[i]>=0.5 and y_label[i]==0): acc-=1
        elif (y_pred[i]<0.5 and y_label[i]==0): acc+=1
        #elif (y_pred[i]<0.5 and y_label[i]==1): acc-=1
    acc = acc / len(y_pred)
    return round(100*(acc),2)
    
def predict_songs(start,end):
    sum_acc1 = 0
    sum_acc2 = 0
    it = 1
    songs = 0
    start = 2*start
    end = 2*end - start
    text = True
    unk_instr = False
    for dir in dir_array:
        for f in os.listdir(dir):
    
            if (start!=0):
                start = start -1
                continue
    
            if (unk_instr):
                unk_instr = False
                continue
        
            if (text):
                y_test = []
                x = open(dir+f)
                for line in x.readlines():
                    y_test.append(line[:3])
                    if (not (line[:3] in instr)): 
                        unk_instr = True
                        #print("Unknown instrument in: ",f)
                        #print("")
                if (unk_instr): 
                    continue
                y_label = to_categorical([y_test])
        
            if (not text):
                predict = []
                wav,sr = librosa.load(dir+f,mono=True,sr=22050)
                wav = wav / np.sqrt(np.mean(wav**2))
                chunks = make_chunks(wav,sr)
                for ch in chunks[:-1]:
                    #mel = librosa.feature.melspectrogram(ch,n_mels=96,fmax=11025,n_fft=1024,hop_length=256,power=1)
                    mel = librosa.core.cqt(ch,n_bins=96,hop_length=256)
                    mel = librosa.core.amplitude_to_db(np.abs(mel))
                    predict.append(mel)
                predict = np.expand_dims(np.array(predict),-1)
                y_pred = prediction(predict)
        
                acc1 = average_accuracy(y_pred,y_label)
                acc2 = existance_accuracy(y_pred,y_label)
            
                sum_acc1 = sum_acc1 + acc1
                sum_acc2 = sum_acc2 + acc2
                songs = songs + 1

                print("Predicted track    : ",f)
                print("Expected output    : ", y_label)
                print("Predicted output   : ", y_pred)
                print("Percentage Accuracy: ", acc1,"%")
                print("Existance Accuracy : ", acc2,"%")
                print("")
            
            
            if (it == end): break
            it = it + 1
            text = not text
    return [sum_acc1,sum_acc2,songs]

In [98]:
start = 1
end = 5000
[sum_acc1,sum_acc2,songs] = predict_songs(start,end)

aver_acc1 = round(sum_acc1/songs,2)
aver_acc2 = round(sum_acc2/songs,2)

print("------------------------------------------------------")
print("Average Percentage Accuracy among",songs,"songs: ",aver_acc1,"%")
print("Average Existance  Accuracy among",songs,"songs: ",aver_acc2,"%")

C:\Users\AgelosK\Anaconda3\lib\site-packages\librosa\core\spectrum.py:983: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  warnings.warn('amplitude_to_db was called on complex input so phase '


Predicted track    :  01 - Chet Baker - Prayer For The Newborn-11.wav
Expected output    :  [1, 0]
Predicted output   :  [0.87 0.32]
Percentage Accuracy:  77.5 %
Existance Accuracy :  100.0 %

Predicted track    :  01 - Chet Baker - Prayer For The Newborn-12.wav
Expected output    :  [1, 0]
Predicted output   :  [0.79 0.45]
Percentage Accuracy:  67.0 %
Existance Accuracy :  100.0 %

Predicted track    :  01 - Chet Baker - Prayer For The Newborn-17.wav
Expected output    :  [1, 0]
Predicted output   :  [0.92 0.62]
Percentage Accuracy:  65.0 %
Existance Accuracy :  50.0 %

Predicted track    :  01 - Chet Baker - Prayer For The Newborn-18.wav
Expected output    :  [1, 0]
Predicted output   :  [0.84 0.56]
Percentage Accuracy:  64.0 %
Existance Accuracy :  50.0 %

Predicted track    :  01 - Da Duh Dah-1.wav
Expected output    :  [1, 0]
Predicted output   :  [1.   0.12]
Percentage Accuracy:  94.0 %
Existance Accuracy :  100.0 %

Predicted track    :  01 - Da Duh Dah-10.wav
Expected output   

Predicted track    :  01 - Roadgame-7.wav
Expected output    :  [1, 0]
Predicted output   :  [0.86 0.39]
Percentage Accuracy:  73.5 %
Existance Accuracy :  100.0 %

Predicted track    :  01 - Roadgame-8.wav
Expected output    :  [1, 0]
Predicted output   :  [0.91 0.5 ]
Percentage Accuracy:  70.5 %
Existance Accuracy :  50.0 %

Predicted track    :  01 - Roadgame-9.wav
Expected output    :  [1, 0]
Predicted output   :  [0.88 0.56]
Percentage Accuracy:  66.0 %
Existance Accuracy :  50.0 %

Predicted track    :  01 - So What-1.wav
Expected output    :  [1, 0]
Predicted output   :  [1.   0.18]
Percentage Accuracy:  91.0 %
Existance Accuracy :  100.0 %

Predicted track    :  01 - So What-24.wav
Expected output    :  [1, 0]
Predicted output   :  [1.   0.58]
Percentage Accuracy:  71.0 %
Existance Accuracy :  50.0 %

Predicted track    :  01 - So What-25.wav
Expected output    :  [1, 0]
Predicted output   :  [1.   0.29]
Percentage Accuracy:  85.5 %
Existance Accuracy :  100.0 %

Predicted trac

Predicted track    :  01 Julian Bream - Albeniz- Mallorca, Opus 202-20.wav
Expected output    :  [0, 1]
Predicted output   :  [0.44 0.68]
Percentage Accuracy:  62.0 %
Existance Accuracy :  100.0 %

Predicted track    :  01 Julian Bream - Albeniz- Mallorca, Opus 202-21.wav
Expected output    :  [0, 1]
Predicted output   :  [0.53 0.65]
Percentage Accuracy:  56.0 %
Existance Accuracy :  50.0 %

Predicted track    :  01 Julian Bream - Albeniz- Mallorca, Opus 202-3.wav
Expected output    :  [0, 1]
Predicted output   :  [0.6  0.62]
Percentage Accuracy:  51.0 %
Existance Accuracy :  50.0 %

Predicted track    :  01 Julian Bream - Albeniz- Mallorca, Opus 202-4.wav
Expected output    :  [0, 1]
Predicted output   :  [0.47 0.78]
Percentage Accuracy:  65.5 %
Existance Accuracy :  100.0 %

Predicted track    :  01 Julian Bream - Albeniz- Mallorca, Opus 202-5.wav
Expected output    :  [0, 1]
Predicted output   :  [0.5  0.83]
Percentage Accuracy:  66.5 %
Existance Accuracy :  50.0 %

Predicted track 

Predicted track    :  01. It Don't Mean a Thing (If It Ain't Got That Swing)-13.wav
Expected output    :  [1, 0]
Predicted output   :  [1. 0.]
Percentage Accuracy:  100.0 %
Existance Accuracy :  100.0 %

Predicted track    :  01. It Don't Mean a Thing (If It Ain't Got That Swing)-2.wav
Expected output    :  [1, 0]
Predicted output   :  [1.   0.03]
Percentage Accuracy:  98.5 %
Existance Accuracy :  100.0 %

Predicted track    :  01. It Don't Mean a Thing (If It Ain't Got That Swing)-3.wav
Expected output    :  [1, 0]
Predicted output   :  [0.99 0.13]
Percentage Accuracy:  93.0 %
Existance Accuracy :  100.0 %

Predicted track    :  01. It Don't Mean a Thing (If It Ain't Got That Swing)-4.wav
Expected output    :  [1, 0]
Predicted output   :  [0.98 0.14]
Percentage Accuracy:  92.0 %
Existance Accuracy :  100.0 %

Predicted track    :  01. It Don't Mean a Thing (If It Ain't Got That Swing)-5.wav
Expected output    :  [1, 0]
Predicted output   :  [1.  0.1]
Percentage Accuracy:  95.0 %
Exist

...

Predicted track    :  Massimiliano Morabito - 02_Pizzica Pizzica di Ostuni-17.wav
Expected output    :  [0, 1]
Predicted output   :  [1.   0.41]
Percentage Accuracy:  20.5 %
Existance Accuracy :  0.0 %

Predicted track    :  Massimiliano Morabito - 02_Pizzica Pizzica di Ostuni-2.wav
Expected output    :  [0, 1]
Predicted output   :  [1.   0.86]
Percentage Accuracy:  43.0 %
Existance Accuracy :  50.0 %

Predicted track    :  Massimiliano Morabito - 02_Pizzica Pizzica di Ostuni-5.wav
Expected output    :  [0, 1]
Predicted output   :  [1.   0.91]
Percentage Accuracy:  45.5 %
Existance Accuracy :  50.0 %

Predicted track    :  Massimiliano Morabito - 02_Pizzica Pizzica di Ostuni-9.wav
Expected output    :  [0, 1]
Predicted output   :  [0.99 0.98]
Percentage Accuracy:  49.5 %
Existance Accuracy :  50.0 %

Predicted track    :  nine inch nails - the fragile cd1 - 12 - the great below-3.wav
Expected output    :  [0, 1]
Predicted output   :  [0.85 0.03]
Percentage Accuracy:  9.0 %
Existance Ac

Predicted track    :  02. Yesterdays-15.wav
Expected output    :  [1, 0]
Predicted output   :  [0.98 0.  ]
Percentage Accuracy:  99.0 %
Existance Accuracy :  100.0 %

Predicted track    :  02. Yesterdays-2.wav
Expected output    :  [1, 0]
Predicted output   :  [1. 0.]
Percentage Accuracy:  100.0 %
Existance Accuracy :  100.0 %

Predicted track    :  02. Yesterdays-3.wav
Expected output    :  [1, 0]
Predicted output   :  [1. 0.]
Percentage Accuracy:  100.0 %
Existance Accuracy :  100.0 %

Predicted track    :  02. Yesterdays-4.wav
Expected output    :  [1, 0]
Predicted output   :  [1. 0.]
Percentage Accuracy:  100.0 %
Existance Accuracy :  100.0 %

Predicted track    :  03 - Symptom of the Universe-18.wav
Expected output    :  [0, 1]
Predicted output   :  [0.77 0.98]
Percentage Accuracy:  60.5 %
Existance Accuracy :  50.0 %

Predicted track    :  03 - Symptom of the Universe-19.wav
Expected output    :  [0, 1]
Predicted output   :  [0.41 0.97]
Percentage Accuracy:  78.0 %
Existance Accu

Predicted track    :  05 - Sonata in A minor, Op. post. 143 D.784 - I. Allegro giusto-19.wav
Expected output    :  [1, 0]
Predicted output   :  [1. 0.]
Percentage Accuracy:  100.0 %
Existance Accuracy :  100.0 %

Predicted track    :  05 - Sonata in A minor, Op. post. 143 D.784 - I. Allegro giusto-2.wav
Expected output    :  [1, 0]
Predicted output   :  [1. 0.]
Percentage Accuracy:  100.0 %
Existance Accuracy :  100.0 %

Predicted track    :  05 - Sonata in A minor, Op. post. 143 D.784 - I. Allegro giusto-20.wav
Expected output    :  [1, 0]
Predicted output   :  [1.   0.01]
Percentage Accuracy:  99.5 %
Existance Accuracy :  100.0 %

Predicted track    :  05 - Sonata in A minor, Op. post. 143 D.784 - I. Allegro giusto-21.wav
Expected output    :  [1, 0]
Predicted output   :  [1.   0.04]
Percentage Accuracy:  98.0 %
Existance Accuracy :  100.0 %

Predicted track    :  05 - Sonata in A minor, Op. post. 143 D.784 - I. Allegro giusto-22.wav
Expected output    :  [1, 0]
Predicted output   : 

Predicted track    :  05 achron - hebrew melody-4.wav
Expected output    :  [1, 0]
Predicted output   :  [0.97 0.21]
Percentage Accuracy:  88.0 %
Existance Accuracy :  100.0 %

Predicted track    :  05 achron - hebrew melody-5.wav
Expected output    :  [1, 0]
Predicted output   :  [1. 0.]
Percentage Accuracy:  100.0 %
Existance Accuracy :  100.0 %

Predicted track    :  05 achron - hebrew melody-6.wav
Expected output    :  [1, 0]
Predicted output   :  [1. 0.]
Percentage Accuracy:  100.0 %
Existance Accuracy :  100.0 %

Predicted track    :  05 achron - hebrew melody-7.wav
Expected output    :  [1, 0]
Predicted output   :  [1. 0.]
Percentage Accuracy:  100.0 %
Existance Accuracy :  100.0 %

Predicted track    :  05 achron - hebrew melody-8.wav
Expected output    :  [1, 0]
Predicted output   :  [1. 0.]
Percentage Accuracy:  100.0 %
Existance Accuracy :  100.0 %

Predicted track    :  05 Mendelssohn - Piano Concerto No. 2 - II. Adagio (molto sostenuto)-10.wav
Expected output    :  [1, 0]


Predicted track    :  07-My Romance-4.wav
Expected output    :  [0, 1]
Predicted output   :  [0.59 0.36]
Percentage Accuracy:  38.5 %
Existance Accuracy :  0.0 %

Predicted track    :  07-My Romance-5.wav
Expected output    :  [0, 1]
Predicted output   :  [0.63 0.38]
Percentage Accuracy:  37.5 %
Existance Accuracy :  0.0 %

Predicted track    :  07-My Romance-6.wav
Expected output    :  [0, 1]
Predicted output   :  [0.8  0.21]
Percentage Accuracy:  20.5 %
Existance Accuracy :  0.0 %

Predicted track    :  07-My Romance-7.wav
Expected output    :  [0, 1]
Predicted output   :  [0.36 0.67]
Percentage Accuracy:  65.5 %
Existance Accuracy :  100.0 %

Predicted track    :  07-My Romance-8.wav
Expected output    :  [0, 1]
Predicted output   :  [0.63 0.41]
Percentage Accuracy:  39.0 %
Existance Accuracy :  0.0 %

Predicted track    :  07-My Romance-9.wav
Expected output    :  [0, 1]
Predicted output   :  [0.52 0.42]
Percentage Accuracy:  45.0 %
Existance Accuracy :  0.0 %

Predicted track    :